In [3]:
import os, sys

notebook_path = os.path.dirname(os.path.abspath("__file__"))
# the code path is two folders up from this notebook + /code
core_path = os.path.dirname(notebook_path)
basepath = os.path.dirname(os.path.dirname(notebook_path))

sys.path.append(core_path)
sys.path.append(basepath)

from core.KlustaFunctions import session_analyzable, find_tetrodes, klusta
from core.Tint_Matlab import get_setfile_parameter
from core.defaultParameters import get_default_settings, calculateUseFeatures

# Define Settings

In [4]:
settings = get_default_settings()

# the default naming structure is the directory\session_basename_tetrodeNumber.cut
# for example: C:\Example\ExampleSession_1.cut for tetrode 1.
# However, there may be a cause where you want to re-analyze with different cut file nomenclature
# so we will allow you to append a value to the end to produce the following:
# directory\session_basename[appendValue]_tetrodeNumber.cut
# so if we append 'TEST' the output would be: C:\Example\ExampleSessionTEST_1.cut for tetrode 1
# set the cut_append value to None for the default
cut_append='KK'  # custom append value
# cut_append=None  # uncomment this for default value

# channels to use:
settings['1'] = 1  # defaults to 1, which is on
settings['2'] = 1  # defaults to 1, which is on
settings['3'] = 1  # defaults to 1, which is on
settings['4'] = 1  # defaults to 1, which is on

# features to use:
settings['PC1'] = 1  # defaults to 1, which is on
settings['PC2'] = 1  # defaults to 1, which is on
settings['PC3'] = 1  # defaults to 1, which is on
settings['PC4'] = 0  # defaults to 0, which is off
settings['A'] = 0  # defaults to 0, which is off
settings['Vt'] = 0  # defaults to 0, which is off
settings['P'] = 0  # defaults to 0, which is off
settings['T'] = 0  # defaults to 0, which is off
settings['tP'] = 0  # defaults to 0, which is off
settings['tT'] = 0  # defaults to 0, which is off
settings['En'] = 0  # defaults to 0, which is off
settings['Ar'] = 0  # defaults to 0, which is off

# re-caculating the feature masks (if channel 2 is off, we will make sure to ignore
# those features)
UseFeatures, numFeats = calculateUseFeatures(settings)
settings['UseFeatures'] = UseFeatures  # feature mask
settings['NumFeat'] = numFeats  # number of features used

# Advanced KlustaKwik Parameters
settings['MaxPos'] = 30  # default 3
settings['nStarts'] = 1  # default = 1
settings['RandomSeed'] = 1  # default = 1
settings['DistThresh'] = 6.907755  # default is 6.907755
settings['FullStepEvery'] = 20  # default is 20
settings['ChangedThresh'] = 0.05  # default is 0.05
settings['MaxIter'] = 500  # default is 500
settings['SplitEvery'] = 40  # default is 40
settings['Subset'] = 1 # default is 1
settings['PenaltyK'] = 1.0  # default is 1.0
settings['PenaltyKLogN'] = 0.0  # default is 0.0

# Reporting Values
settings['Verbose'] = 1  # default of 1
settings['Screen'] = 1  # default of 1
settings['Log File'] = 1  # default of 1


# Misc Settings
settings['Silent'] = 0   # the KK windows will pop up
# number of simultaneous threads to analyze, defaults to 1, uncomment to override
settings['NumThreads'] = 4  # default is 1
settings['Cores'] = os.cpu_count()


# Experimenter Settings
experimenter_settings = {
    'Geoff' : 'geoffrey.m.barrett@gmail.com' # can do [email1@.., email2@..] if you want it sent to more than 1
}

# Email Settings
smtp_settings = {}
smtp_settings['Notification'] = 0 #  1 for send e-mails, 0 for don't send

# if you have the notifications set to 0 you don't have to worry about this.
# we will need an e-mail to send these experimenter's e-mails from
smtp_settings['Username'] = 'example@gmail.com'
smtp_settings['Password'] = 'password'  # associated password
smtp_settings['ServerName'] = 'smtp.gmail.com' # the smtp server name, 'smtp.gmail.com' for gmail
smtp_settings['Port'] = 587  # 587 default for gmail

In [5]:
settings

{'1': 1,
 '2': 1,
 '3': 1,
 '4': 1,
 'PC1': 1,
 'PC2': 1,
 'PC3': 1,
 'PC4': 0,
 'A': 0,
 'Vt': 0,
 'P': 0,
 'T': 0,
 'tP': 0,
 'tT': 0,
 'En': 0,
 'Ar': 0,
 'Verbose': 1,
 'Screen': 1,
 'Log File': 1,
 'UseFeatures': '1111111111111',
 'NumFeat': 3,
 'NumThreads': 4,
 'MaxPos': 30,
 'nStarts': 1,
 'RandomSeed': 1,
 'DistThresh': 6.907755,
 'FullStepEvery': 20,
 'ChangedThresh': 0.05,
 'MaxIter': 500,
 'SplitEvery': 40,
 'Subset': 1,
 'PenaltyK': 1.0,
 'PenaltyKLogN': 0.0,
 'Silent': 0,
 'Cores': 12,
 'nonbatch': 0}

# Analyze Directory

In [6]:
batch = False

if batch:
    settings['nonbatch'] = 0
else:
    settings['nonbatch'] = 1

directory = r'E:\Apollo_D_Drive\data\VirtualMazeData\ANT1_2'
directory = r'C:\Users\Taub Institute\Desktop\test_batchtint\small_sample2'

# Finds Sessions

In [7]:
if batch:
    sub_directories = [d for d in os.listdir(directory)
                                   if os.path.isdir(os.path.join(directory, d)) and
                                   d not in ['Processed', 'Converted']]  # finds the subdirectories within each folder

else:
    sub_directories = [os.path.basename(directory)]
    directory = os.path.dirname(directory)

if len(sub_directories) == 0:
    set_files = []
    
for d in sub_directories:
    set_files = [os.path.join(directory, d, file) for file in os.listdir(os.path.join(directory, d))
                             if '.set' in file and
                             not os.path.isdir(os.path.join(directory, d, file))]



In [8]:
print('There are %d total set files found!' % (len(set_files)))

There are 1 total set files found!


In [9]:
# checking that the set files are analyzable (have the appropriate files, or don't have
# existing cut values already. Essentially all it needs are the tetrode files.
if set_files:
    set_files = [file for file in set_files if session_analyzable(os.path.dirname(file), 
                                                             os.path.splitext(
                                                                 os.path.basename(file))[0],
                                                            append=cut_append)]

In [10]:
print('There are %d un-analyzed set files found!' % (len(set_files)))

There are 0 un-analyzed set files found!


In [11]:
analyzed_set_files = klusta(set_files, settings, 
       smtp_settings=smtp_settings, 
       experimenter_settings=experimenter_settings,
                           append=cut_append)